## Setting up the Variables

Call Stock Price after $n$ periods of time $S_n$. Call price of a derivative security, in this case a European Call Option, after $n$ periods of time $V_n$. We will define the initial stock price as $S_0 = 4$. The experiation time of the option will be defined as $25$. The goal will be to find $V_0$,  the price of the option at time $0$.

In [1]:
S_0 = 4

The price of the option at time $25$ will be defined as $V_{25} = (S_{25} - K)^+$, the max of $S_{25}$ minus the strike price, $K$, and $0$. This means that if the option expires with $S_{25}$ above the strike price you will take home the difference but if $S_{25}$ is less than the strike price at expiration the option expires worthless.

In [2]:
def option_expiration_value(stock_price, strike_price):
    return max(stock_price - strike_price, 0)

For this we will say $K = 10$.

In [3]:
K = 5

As this model is based on a bionmial model, every time period will be thought of as a random coin flip resulting in heads, $H$, or tails, $T$. For this model risk-neutral probabilities will dictate whether the coin lands on $H$ or $T$. The risk-neutral probability for $H$ will be $\tilde{p}$ and the risk-neutral probability for $T$ will be $\tilde{q}$. They will hold the condition $\tilde{p} + \tilde{q} = 1$. But, these probabilities will not be constant, they will also change depending on whether or not the coin flip results in $H$ or $T$. If the result is $H$ $\tilde{p_n}$ will be multiplied by $x$ and $\tilde{q_n}$ will be redefined as $1 - \tilde{p_n}$. If the result is $T$ the same will be done with $y$ instead of $x$. The probability of $H$ at time $0$ will be defined as $\tilde{p_0} = \frac{11}{20}$ or $55\%$ and the probability of $T$ at time $0$ will be defined as $\tilde{q_0} = \frac{9}{20}$ or $45\%$. Variables $x$ and $y$ will be defined as $x = \frac{1001}{1000}$ and $y = \frac{999}{1000}$. This will make the affected values go up $0.1\%$ if the result is $H$ and go down $0.1\%$ if the result is $T$.

In [4]:
p_0 = 11/20
q_0 = 9/20
x = 1001/1000
y = 999/1000

In this model the the amount the stock price increases, the up factor, $u_n$, and the amount the stock price decreases, the down factor, $d_n$, will also be variable rates. As the variables above if the the result is $H$ $u_n$ and $d_n$ will be multiplied by $x$, and if the result is $T$ they will be multiplied by $y$. If we define number of $H$ as $N_H$ and number of tails as $N_T$, $u_n = \frac{S_1(H)}{S_0}x^{N_H}y^{N_T}$ and $d_n = \frac{S_1(T)}{S_0}x^{N_H}y^{N_T}$.

We will define $u_0 = 2$ and $d_0 = 1/2$.

In [5]:
u_0 = 2
d_0 = 1/2

In this model we will also have a variable interest rate, $r_n$, that is also multiplied by $x$ or $y$ depending on if the coin toss results in $H$ or $T$. The interest rate at time $0$ will be defined as $r_0 = 1/4$.

In [6]:
r_0 = 1/4

## Option Pricing

We will get the price of the option at time $0$ using the recursive formula $V_n = \frac{1}{1 + r_n}[\tilde{p_n}V_{n+1}(H) + \tilde{q_n}V_{n+1}(T)]$. We will loop through every possible of $H$ and $T$ across the $25$ time periods and recur back through this formula until we are left with one price, $V_0$, the initial price of the option at time $0$.

Below I will show how we will get all possible combinations for $H$ and $T$ at time $n$. But, this is pretty simple because **the order of $H$ and $T$ does not matter.**

In [7]:
def get_combinations(n):
    combinations = []
    for i in range(n + 1):
        combo = 'H' * i + 'T' * (n - i)
        combinations.append(combo)
    return combinations

print(get_combinations(10))

['TTTTTTTTTT', 'HTTTTTTTTT', 'HHTTTTTTTT', 'HHHTTTTTTT', 'HHHHTTTTTT', 'HHHHHTTTTT', 'HHHHHHTTTT', 'HHHHHHHTTT', 'HHHHHHHHTT', 'HHHHHHHHHT', 'HHHHHHHHHH']


Next, we need to set up the function for the recursive formula stated above.

In [8]:
def price_option_recursion(r, p, q, price_h, price_t):
    return (1/(1+r)) * (p * price_h + q * price_t)

After this I can set up the recursive function to use `price_option_recursion` which was defined above. The parameters `start_price`, `strike_price`, `s`, `u`, and `d` can be defaulted to `None` as they are only needed for the first recursion. The parameter `price_dict` can also be defaulted to `None` as it has no value when first calling the function.

In [9]:
def get_call_option_price(expiration_t, p, q, r, input_x, input_y, start_price=None, strike_price=None, u=None, d=None, price_dict=None):
    if expiration_t == 1:
        return (1/(1+r)) * (p * price_dict['H'] + q * price_dict['T'])
    
    combinations = get_combinations(expiration_t - 1)
    new_price_dict = {}
    
    if price_dict == None:
        for combo in combinations:
            s_n, p_n, q_n, u_n, d_n, r_n = start_price, p, q, u, d, r
            for outcome in combo:
                if outcome == 'H':
                    s_n *= u_n
                    p_n *= input_x
                    q_n = 1 - p_n
                    u_n *= input_x
                    d_n *= input_x
                    r_n *= input_x
                elif outcome == 'T':
                    s_n *= d_n
                    p_n *= input_y
                    q_n  = 1 - p_n
                    u_n *= input_y
                    d_n *= input_y
                    r_n *= input_y
                new_price_dict[combo] = price_option_recursion(r_n, p_n, q_n, option_expiration_value(s_n * u_n, strike_price), option_expiration_value(s_n * d_n, strike_price))
    else:
        for combo in combinations:
            p_n, q_n, r_n = p, q, r
            for outcome in combo:
                if outcome == 'H':
                    p_n *= input_x
                    q_n *= input_x
                    r_n *= input_x
                elif outcome == 'T':
                    p_n *= input_y
                    q_n *= input_y
                    r_n *= input_y
            new_price_dict[combo] = price_option_recursion(r_n, p_n, q_n, price_dict['H' + combo], price_dict[combo + 'T'])
    return(get_call_option_price(expiration_t - 1, p, q, r, input_x, input_y, price_dict=new_price_dict))
        

And now we can call `get_call_option_price` with all the parameters to get the value of $V_0$.

In [10]:
get_call_option_price(25, p_0, q_0, r_0, x, y, S_0, K, u_0, d_0)

26.36645993406144